In [24]:
import pandas as pd
import numpy as np
import plotly.express as px
import contractions
import pickle
import nltk
from nltk.corpus import words, stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from textblob import TextBlob

In [25]:
# Now we download a dataset of all possible words in the English language from ltk and a list of stopwords
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


duden = np.array(words.words())
stopwords = np.array(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package words to /home/jon/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/jon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
comments = pd.read_csv("./bank-data/bank-comments.csv", sep="\t")
customers = pd.read_csv("./bank-data/bank-tabular.csv", sep="\t")

# Data Cleaning

When looking at the data, we can see that there are some issues with the data. We will try to fix these issues in this notebook.

The issues directly apparent are Lots of NaN values, inconsistent formatting of tags and lots of spelling mistakes.

So this is our todo list for cleaning the data:

## Issues with the customers

- [X] non structured gender names
  - [X] Male / Female
  - [X] m/f
  - [X] Unspecified
  - [X] NaN
  - [X] Not Specified
- [~] NaN values (we will address those during the analysis)
- [X] Normalization of the age column
## Issues with the comments:

- [X] NaN values in the comments column
- [X] special characters in the comments column like asterisks, umlaute, brackets, etc.
- [X] Spelling mistakes in the comments column


> Side note: Depending on how and what we analyse in the tabular data we will need to fill up the NaN values differently. For the time being we will leave them as they are and address them during the analysis.

## Cleaning the comments

In order to be able to do a nice analysis of the comments, we need to clean them up a bit. 

We will do this by:

1. dropping all rows with NaN values in the comments column as they hold little value for this part of the analysis
2. expanding contractions and removing special characters
    - before we remove all non alphanumeric characters, we want to expand contractions like "don't" to "do not".
    - then we remove all non alphanumeric characters which makes it easier to work with the text.
4. we then iliminate words with little value for the analysis.
    1. first the generally low content words, fillers like "and", "the", "a", "an", etc.
    2. then we remove words which have a high frequency in our dataset, but are not very informative for the analysis. These are words like "bank", "customer".
5. after we have removed all the words with little value, we can stem the remaining words. This will reduce the number of unique words in our dataset, which will make the analysis easier.
   1. we start of by removing the suffixes of the words, like "ing", "ed", "s", etc.
   2. we try to fix spelling mistakes by looking at the words which are close to the word we are trying to fix. We then replace the word with the most similar word.
   

In [27]:
# Removing rows with no comments
# Because the timestamps of empy comments hold no value for us we can simply remove them
# This reduces the size of the dataset and makes it easier to work with

comments = comments[comments["comments"].notnull()]


# Before we remove special characters like umlauts and brackets we need to make sure we do not lose too much information
# A good example for this is the ' which is an indicator for a contraction like "don't" or "can't" being shortened to "do not / can not"
# In order to make shure we do not lose this information we first expand the contractions and then remove the special characters
# we can do this by simply utilizing the contractions library
comments['comments'] = comments["comments"].apply(lambda x: contractions.fix(x)) 
# Now we are free to remove all characters that are not letters, numbers or spaces
comments['comments'] = comments["comments"].str.replace('[^a-zA-Z0-9\s]', '', regex=True)


# And now we make them all lowercase in order to reduce the amount of unique words which are semantically the same
comments['comments'] = comments["comments"].str.lower()

# To compare the effectiveness of our cleaning we will save this sample and compare it to the cleaned version later
datasets = {}
datasets["lower"] = comments

In [28]:
# Now we address the issue of the many spelling mistakes in the comments
# We can do this by using the textblob library which is a library for processing textual data
# it has a function called correct() which will correct the spelling of a word

comments['comments'] = comments["comments"].apply(lambda x: str(TextBlob(x).correct()))

# we also save this cleaned dataset as a pickle file
datasets["spelling"] = comments

In [29]:
# The next step is to remove all stopwords
# This means we remove low value words like "the", "a", "an" etc. which do not add any actual meaning to the sentence.
# We do this by taking every word of every comment and checking if it is in the list of stopwords provided by nltk

# Here is an example of what stopwords are in our dataset:
print(comments['comments'].apply(lambda x: [item for item in x.split() if item in stopwords])[:5])

# Now we can remove them
comments['comments'] = comments["comments"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

datasets["stopwords"] = comments

0                                           [this, is]
1        [to, the, s, and, all, in, all, m, with, the]
2                                     [is, in, a, for]
6                                              [to, a]
8    [i, that, my, is, to, and, its, the, are, too,...
Name: comments, dtype: object


In [30]:
# Now that we have removed all unessesary words we can lemmatize the remaining words which further reduces the amount of unique words
# This is done by reducing words to their root form
# So words like "running" and "ran" are reduced to "run" and "better" and "best" are reduced to "good"


comments['comments'] = comments["comments"].apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))
datasets["lemmatize"] = comments

## Inconsistency in the customer_gender tags

As we can see by the following query, there are 6 different tags for the genders. \
Three of those are semantically the same, but are written differently.

A simple solution is to replace the redundant tags with their counterparts.\
Therefore we replace the following tags:

- "Female" -> f
- "Male" -> m
- "Not Specified", "Unspecified" -> n

Maybe the "n" tag should be replaced by a simple NaN value, because for this analysis they are functionally the same. \
But for the sake of completeness, we will keep them.

In [31]:
customers["customer_gender"].value_counts()

customer_gender
Female           1124
Male             1057
Not specified     221
f                 134
m                 118
Unspecified        34
Name: count, dtype: int64

In [32]:
customers["customer_gender"].replace({"Female":"f","Male":"m","Not specified": 'n', "Unspecified": 'n'}, inplace=True)

## Normalization of the age column

As we will see during the analysis, the age column is not very well normalized. \
For example ther are no values lower than 18 and no values higher than 100. \
But because we still want to  use the non normalized values for some visualizations, we will keep the original values but also add a normalized column.

For normalization we will use the following formula:

$\LARGE \frac{x - min(x)}{max(x) - min(x)}$

This will normalize the values between 0 and 1. 

In [33]:
customers["customer_age_norm"] = customers["customer_age"].apply(lambda x: (x - customers["customer_age"].min()) / (customers["customer_age"].max() - customers["customer_age"].min()))

# Saving data

Now that we have done some preprocessing we can save the data to work on it in the analysis notebooks.

In [34]:
# now we can save the cleaned dataframes as a pickle file

comments.to_pickle("./bank-data/cleaned_comments.pkl")
customers.to_pickle("./bank-data/cleaned_customers.pkl")

# and the datasets for the different cleaning steps
with open("./bank-data/cleaned_datasets.pkl", "wb") as f:
    pickle.dump(datasets, f)